Alright. Take a look at the very first example:
```
SELECT
  name, 
  opened,
  LEAD(name) OVER(ORDER BY opened)
FROM website;
```
The analytic function here is `LEAD(name)`. `LEAD` with a single argument in the parentheses looks at the next row in the given order and shows the value in the column specified as the argument.

For all the statistics of the website with id = 1, show the day, the number of users and the number of users on the next day.
```
select
	day, users,
    LEAD(users) over(order by day)
from statistics
where website_id = 1
```

`LEAD` can be extremely useful when we want to calculate deltas, i.e. differences between two values. A typical example may look like this:
```
SELECT
  day,
  clicks,
  LEAD(clicks) OVER(ORDER BY day),
  clicks - LEAD(clicks) OVER(ORDER BY day) 
FROM statistics
WHERE website_id = 2;
```
The above query calculates day-to-day deltas: the last column shows the difference in clicks between the current day and the next day. From a business point of view, this could easily tell us a lot about the website: if the deltas for many rows are positive, and possibly increasing, then the website is expanding. If, in turn, the deltas are mostly negative, we can start to worry about the performance of the website.

For `website_id = 1`, show each statistics row: day, revenue, revenue on the next day and the difference between these two values.
```
select
	day, revenue,
    LEAD(revenue) over(order by day),
    revenue - LEAD(revenue) over(order by day)
from statistics
where website_id = 1
```

Fantastic! There is also another version of `LEAD`. It takes two arguments: `LEAD(x,y)`. `x` remains the same – it specifies the column to return. `y`, in turn, is a number which defines the number of rows forward from the current value. For instance:
```
SELECT
  name,
  opened,
  LEAD(opened,2) OVER(ORDER BY opened)
FROM website;
```
This form of `LEAD` won't show the webpage with the opening date coming immediately after the current opening date. Instead, it will show the opening date 2 rows forward – the 1st row will show the 3rd date, etc.

Take the statistics for the website with `id = 2` between 1 and 14 May 2016 and show the day, the number of users and the number of users 7 days later.

Note that the last 7 rows don't have a value in the last column, because no rows '7 days from now' can be found for them.
```
select
	day, users,
    LEAD(users, 7) over(order by day)
from statistics
where website_id = 2
and day >= '2016-05-01' and day <= '2016-05-14'
```

The last possible type of `LEAD` takes three arguments:
```
SELECT
  name,
  opened,
  LEAD(opened,2,'2000-01-01') OVER(ORDER BY opened)
FROM website;
```
The new (last) argument tells the function what it should return if no matching value is found. Previously, the last rows got NULLs from the function, because there were no "lead" (further) rows for them. Now, you can specify what should be displayed in such cases instead of the default NULL. Here, we show '2000-01-01'. Note that this value must be of the same type as the column itself: if you show dates with LEAD, the last argument must be a date too. You can't show "not available" or 0 instead.

Of course, there's also a function that shows a previous value, and its name is `LAG(x)`:
```
SELECT
  name,
  opened,
  LAG(name) OVER(ORDER BY opened)
FROM website;
```
Now, instead of showing the next opening date, we show the previous opening date

Show the statistics for the website with `id = 3`: day, number of clicks that day and the number of clicks on the previous day.

Note that there won't be any previous value for the first row.
```
select
	day, clicks, 
    LAG(clicks) over(order by day)
from statistics
where website_id = 3
```

`RPM` (revenue per thousand impressions) is defined as the revenue divided by the number of impressions, times 1000.
```
RPM = (revenue / number of impressions) * 1000
```
For each statistics row with `website_id = 2`, show the day, the RPM and the RPM 7 days later. Rename the columns to `RPM` and `RPM_7`.

```
select
	day,
    (revenue / impressions) * 1000 as RPM,
    (LEAD(revenue, 7) over(order by day) / LEAD(impressions, 7) over(order by day)) * 1000 as RPM_7
from statistics
where website_id = 2
```

Let's define conversion rate as the number of clicks divided by the number of impressions, multiplied by 100 to obtain percentages.

For `website_id = 1` and dates between May 15 and May 31, show each statistics row: `day`, `clicks`, `impressions`, conversion rate (as the `conversion` column) and the conversion rate on the previous day (as the previous_conversion column).
```
select
	day, clicks, impressions,
    (clicks::numeric / impressions) * 100 as conversion,
    (LAG(clicks::numeric) over(order by day) / LAG(impressions) over(order by day)) * 100 as previous_conversion
from statistics
where website_id = 1
and day >= '2016-05-15' and day <= '2016-05-31'
```

`LEAD` and `LAG` are 2 functions which are always relative to the current row. Now, we'll get to know three other functions that are independent of the current row.

The first one is `FIRST_VALUE(x)`. As you may guess, it returns the first value in the column x in the given order. Take a look:
```
SELECT
  name,
  opened,
  budget,
  FIRST_VALUE(budget) OVER(ORDER BY opened)
FROM website;
```
Here, we still sort rows by the opening date (`ORDER BY opened`), but we show the lowest budget instead of the first opening date (`FIRST_VALUE(budget)`). In this way, we can show the budget for the website that was opened first.

Note that this would be impossible to achieve with a simple `MIN(...)` function. `MIN(budget)` would simply show the smallest budget: 500 in this case. That is not the same as the budget of the first website (3000).

Show the statistics for `website_id = 2`. For each row, show the day, the number of users and the smallest number of users ever.
```
select
	day, users,
    FIRST_VALUE(users) over(order by users)
from statistics
where website_id = 2
```

Show the statistics for `website_id = 3`. For each row, show the day, the revenue and the revenue on the first day.
```
select 
	day, revenue,
    first_value(revenue) over(order by day)
from statistics
where website_id = 3
```

If there is an `ORDER BY` clause, `RANGE UNBOUNDED PRECEDING` will be used as the default window frame.

And this is precisely the cause of our troubles. We indeed used `ORDER BY` within `OVER(...)`, which is why `LAST_VALUE(x)` only considers the rows from the first row until the current row. The solution is quite simple: we need to define the right window frame:
```
SELECT
  name,
  opened,
  LAST_VALUE(opened) OVER(
    ORDER BY opened
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM website;
```

Here's one thing you should remember: while `FIRST_VALUE` works well with the default window frame, `LAST_VALUE` needs an explicit definition of the right window frame to actually make sense.

Of course, you can order by one column and return the other:
```
SELECT
  name,
  budget,
  LAST_VALUE(budget) OVER(
    ORDER BY opened
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM website;
```
Here we show the budget of the most recent website (not the greatest budget).

Show the statistics for `website_id = 1`. For each row, show the day, the number of impressions and the number of impressions on the day with the most users.
```
select 
	day, impressions,
    LAST_VALUE(impressions) over(order by users rows between unbounded preceding and unbounded following)
from statistics
where website_id = 1
```

For each statistics rows with `website_id = 1`, show the day, the number of users, the number of users on the last day and the difference between these two values.
```
select
	day, users,
    LAST_VALUE(users) over(order by day rows between unbounded preceding and unbounded following),
    users - LAST_VALUE(users) over(order by day rows between unbounded preceding and unbounded following)
from statistics
where website_id = 1
```

The last function we'll learn in this part is: `NTH_VALUE(x,n)`. This function returns the value in the column x of the nth row in the given order.
```
SELECT
  name,
  opened,
  NTH_VALUE(opened, 2) OVER(
    ORDER BY opened
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM website;
```
This time, we're showing the opening date of the current row together with the second row when sorted by the opening date. With NTH_VALUE, we also need to redefine the window frame. Otherwise, some rows will display incorrect values.

Just to remind you, you can always revert the order by adding the word `DESC`, which sometimes comes in handy with `NTH_VALUE`:

`...OVER(ORDER BY opened DESC)...`

Take the statistics for the website with id = 2 between May 15 and May 31, 2016. Show the day, the revenue on that day and the third highest revenue in that period.

```
select
	day, revenue,
    NTH_VALUE(revenue, 3) over(order by revenue desc rows between unbounded preceding and unbounded following)
from statistics
where website_id = 2 
and day >= '2016-05-15'
and day <= '2016-05-31'
```

Let's run some cross-website statistics now. Take the day May 14, 2016 and for each row, show: `website_id`, revenue on that day, the highest revenue from any website on that day (`AS highest_revenue` and the lowest revenue from any website on that day (`as lowest_revenue`).
```
select
	website_id, revenue,
    FIRST_VALUE(revenue) over(order by revenue desc rows between unbounded preceding and unbounded following) as highest_revenue,
    FIRST_VALUE(revenue) over(order by revenue rows between unbounded preceding and unbounded following) as lowest_revenue
from statistics
where day = '2016-05-14'
```

Take the statistics for `website_id = 1`. For each row, show the day, the number of clicks on that day and the median of clicks in May 2016 (calculated as the 16th value of all 31 values in the column clicks when sorted by the number of clicks).
```
select
	day, clicks,
    NTH_VALUE(clicks, 16) over(order by clicks rows between unbounded preceding and unbounded following)
from statistics
where website_id = 1
```

For each statistics row of `website_id = 3`, show the day, the number of clicks on that day and a ratio expressed as percentage: the number of clicks on that day to the greatest number of clicks on any day. Round the percentage to integer values.
```
select
	day, clicks,
    ROUND((clicks::numeric / FIRST_VALUE(clicks) over(order by clicks desc rows between unbounded preceding and unbounded following)) * 100)
from statistics
where website_id = 3
```

For each row, show the day, the price on that day and the price on the next day.
```
select
	day, price,
    LEAD(price) over(order by day)
from advertisement
```

For each row, show the day, the price on that day, the price 7 days earlier and the difference between these two values.
```
select
	day, price,
    LAG(price, 7) over(order by day),
    price - LAG(price, 7) over(order by day)
from advertisement
```

For each row, show the day, the price on that day, the highest price ever (column name highest_price) and the lowest price ever (column name lowest_price).
```
select
	day, price,
    LAST_VALUE(price) over(order by price rows between unbounded preceding and unbounded following) as highest_price,
    LAST_VALUE(price) over(order by price desc rows between unbounded preceding and unbounded following) as lowest_price
from advertisement
```